In [ ]:
# =====================================
# BOOTSTRAP TEST WITH REPLACEMENT
# =====================================
# the idea of the code below is to resample the original dataset with replacement
# and calculate in each resample the p-value of the Dunn's post hoc test

import numpy as np
import pandas as pd
from scipy.stats import kruskal
from scikit_posthocs import posthoc_dunn
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

wolbachia_df = pd.read_csv("Wolbachia_propSingle.txt", sep="\t", header=0)

enumerate_ = 1
result_dunn_pValues_Temp = []
result_dunn_pValues_Sex_f = []
for _ in range(10000):
    bootstrap_sample = wolbachia_df.sample(frac=1, replace=True)  # Bootstrap sample with replacement
    KW_test = kruskal(bootstrap_sample['W_Single_prop_Infected'], bootstrap_sample['Temp'], bootstrap_sample['Sex_f'],)
    dunn_result = posthoc_dunn([bootstrap_sample['W_Single_prop_Infected'], bootstrap_sample['Temp'], 
                                bootstrap_sample['Sex_f']], p_adjust='holm')
    result_dunn_pValues_Temp.append(dunn_result.iloc[0,1])  # take the p-value for of the effect of temperature
    result_dunn_pValues_Sex_f.append(dunn_result.iloc[0,2])  # take the p-value for of the effect of sex ratio
    if enumerate_ % 50 == 0:
        print(enumerate_, end=" ")
    enumerate_ += 1

In [ ]:
mean_pvalue_Temp = np.mean(result_dunn_pValues_Temp)
mean_pvalue_Sex_f = np.mean(result_dunn_pValues_Sex_f)

print("Mean Temp p-value:", mean_pvalue_Temp)
print("Mean Sex_f p-value:", mean_pvalue_Sex_f)

In [ ]:
# Calculate the lower 95% confidence limit for Temp
lower_conf_limit = np.percentile(result_dunn_pValues_Temp, 2.5)
upper_conf_limit = np.percentile(result_dunn_pValues_Temp, 97.5)
mean_pvalue = np.mean(result_dunn_pValues_Temp)

print("Low 95% confidence limit - Temp:", lower_conf_limit)
print("High 95% confidence limit - Temp:", upper_conf_limit)
print("Mean of the p-values - Temp:", mean_pvalue)

In [ ]:
# Calculate the lower 95% confidence limit for Sex_f
lower_conf_limit = np.percentile(result_dunn_pValues_Temp, 2.5)
upper_conf_limit = np.percentile(result_dunn_pValues_Temp, 97.5)
mean_pvalue = np.mean(result_dunn_pValues_Temp)

print("Low 95% confidence limit - Temp:", lower_conf_limit)
print("High 95% confidence limit - Temp:", upper_conf_limit)
print("Mean of the p-values - Temp:", mean_pvalue)

In [ ]:
# ==================================
# THE CODE THAT PRODUCES THE FIGURE
# ==================================

import matplotlib.pyplot as plt
import numpy as np

# Apply log transformation to p-values - Temp
log_transformed_pvalues_Temp = -np.log10(result_dunn_pValues_Temp)

# Apply log transformation to p-values - Sex_f
log_transformed_pvalues_Sex_f = -np.log10(result_dunn_pValues_Sex_f)

# Set custom colors
color_temp = 'hotpink'
color_sex_f = 'lightgrey'

# Set figure size with two subplots arranged vertically
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 14), dpi=600)

# Define the threshold for coloring bars in red
threshold = -np.log10(0.05)

# Plot histogram with log-transformed p-values - Temp
hist_temp, bins_temp, patches_temp = ax1.hist(log_transformed_pvalues_Temp, bins=200, edgecolor='snow', color=color_temp)

# Color bars below the threshold in red
for patch, value in zip(patches_temp, bins_temp[:-1]):
    if value > threshold:
        patch.set_facecolor('hotpink')

ax1.set_xlabel(r'Negative log$_{10}$-transformed P-values', fontsize=16)
ax1.set_ylabel('Frequency', fontsize=18)
ax1.set_title(r'Distribution of the simulated negative log$_{10}$-transformed P-values\nfor the effect of temperature', fontsize=18, y=1.08)  # Adjust y value

# Set spines for the first graph
ax1.spines['right'].set_visible(False)
ax1.spines['top'].set_visible(False)
ax1.spines['left'].set_position(('outward', 10))  # Add some distance to the left spine
ax1.spines['bottom'].set_position(('outward', 10))  # Add some distance to the bottom spine

ax1.text(-0.10, 1.20, "A", transform=ax1.transAxes, fontsize=28, fontweight='bold')

# Add vertical lines at the 0.05 level
# ax1.axvline(x=-np.log10(0.05), color='hotpink', linestyle='dashdot', label='Threshold = 0.05', linewidth=3)
ax1.axvline(x=-np.log10(6.498389049914289e-07), color='blue', linestyle='dashdot', label=r'Observed P-value = $6.49 \times 10^{-7}$', linewidth=3)
ax1.axvline(x=-np.log10(4.921172007827087e-08), color='purple', linestyle='dashdot', label=r'Low CI = $4.92 \times 10^{-8}$', linewidth=3)
ax1.axvline(x=-np.log10(1.5422718711843626e-05), color='purple', linestyle='dashdot', label=r'High CI = $1.54 \times 10^{-5}$', linewidth=3)
ax1.legend(bbox_to_anchor=(0.30, 0.95), fontsize="large")

# Plot histogram with log-transformed p-values - Sex_f
hist_sex_f, bins_sex_f, patches_sex_f = ax2.hist(log_transformed_pvalues_Sex_f, bins=200, edgecolor='snow', color=color_sex_f)

# Color bars below the threshold in red
for patch, value in zip(patches_sex_f, bins_sex_f[:-1]):
    if value > threshold:
        patch.set_facecolor('hotpink')

ax2.set_xlabel(r'Negative log$_{10}$-transformed P-values', fontsize=16)
ax2.set_ylabel('Frequency', fontsize=18)
ax2.set_title(r'Distribution of the simulated negative log$_{10}$-transformed P-values\nfor the effect of female:male ratio', fontsize=18, y=1.08)  # Adjust y value

# Set spines for the second graph
ax2.spines['right'].set_visible(False)
ax2.spines['top'].set_visible(False)
ax2.spines['left'].set_position(('outward', 10))  # Add some distance to the left spine
ax2.spines['bottom'].set_position(('outward', 10))  # Add some distance to the bottom spine

ax2.text(-0.10, 1.20, "B", transform=ax2.transAxes, fontsize=28, fontweight='bold')

# Add vertical lines at the 0.05 level
# ax2.axvline(x=-np.log10(0.05), color='hotpink', linestyle='dashdot', label='Threshold = 0.05', linewidth=3)
ax2.axvline(x=-np.log10(0.053212), color='blue', linestyle='dashdot', label='Observed P-value = 0.053', linewidth=3)
ax2.axvline(x=-np.log10(0.008357755617266907), color='purple', linestyle='dashdot', label='Low CI = 0.008', linewidth=3)
ax2.axvline(x=-np.log10(0.5315999878883019), color='purple', linestyle='dashdot', label='High CI = 0.53', linewidth=3)
ax2.legend(bbox_to_anchor=(0.64, 0.77), fontsize="large")

# Adjust layout with vertical space between subplots
plt.subplots_adjust(hspace=0.55)

# Save the entire figure as a PDF file for better quality
plt.savefig("Figure_3_rev_proofs.pdf", format='pdf', bbox_inches='tight')

# Show the entire figure (if you want to display it in the notebook)
plt.show()
